<a href="https://colab.research.google.com/github/gyaneshhere/VoiceAIInterface/blob/main/Building_Voice_Agents_with_FastRTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aimug-org/austin_langchain/blob/main/labs/LangChain_112/Building_Voice_Agents_with_FastRTC.ipynb)  

# Building Voice Agents with FastRTC

This notebook will walk you through setting up and testing a simple FastRTC server.  

## Prerequisites

1. `uv` from [astral.sh](https://docs.astral.sh/uv/) to manage python environments. (Already present in Google Colab)
2. OpenAI API Key from https://platform.openai.com/settings/organization/api-keys available as environment variable `OPENAI_API_KEY` or as Google Colab Secrets
3. (Optional for FastRTC Client WebPage test) Ngrok AuthToken from https://dashboard.ngrok.com/get-started/your-authtoken available as environment variable `NGROK_AUTH_TOKEN` or as Google Colab Secrets

## Initialize UV project and virtual environment

Initialize a new `uv` project with `uv init`.  
Pin Python to version 3.13  
Create a new virtual environment for the project.

Note: `export VIRTUAL_ENV=` is needed only within Jupyter notebooks and can be excluded otherwise.

In [ ]:
%%bash
export VIRTUAL_ENV=
uv init
uv python pin 3.13
uv venv

Updated `.python-version` from `3.11` -> `3.13`


Initialized project `content`
 Downloaded cpython-3.13.2-linux-x86_64-gnu
Using CPython 3.13.2
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate


## Add FastRTC to the project dependencies

Note: `export VIRTUAL_ENV=.venv` is needed only within Jupyter notebooks and can be excluded otherwise.

In [ ]:
%%bash
export VIRTUAL_ENV=.venv
uv add fastrtc[vad,tts,stt]

Resolved 118 packages in 1.41s
 Downloaded soundfile
 Downloaded aiortc
 Downloaded pydantic-core
 Downloaded fastrtc
 Downloaded pylibsrtp
 Downloaded pygments
 Downloaded tokenizers
 Downloaded pillow
 Downloaded cryptography
 Downloaded numba
 Downloaded ruff
 Downloaded babel
 Downloaded sympy
 Downloaded pandas
 Downloaded espeakng-loader
 Downloaded scikit-learn
 Downloaded numpy
 Downloaded onnxruntime
 Downloaded av
 Downloaded llvmlite
 Downloaded hf-xet
 Downloaded scipy
 Downloaded gradio
Prepared 116 packages in 15.21s
Installed 116 packages in 325ms
 + aiofiles==24.1.0
 + aioice==0.10.1
 + aiortc==1.11.0
 + annotated-types==0.7.0
 + anyio==4.9.0
 + attrs==25.3.0
 + audioop-lts==0.2.1
 + audioread==3.0.1
 + av==14.3.0
 + babel==2.17.0
 + certifi==2025.4.26
 + cffi==1.17.1
 + charset-normalizer==3.4.2
 + click==8.2.0
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + colorlog==6.9.0
 + cryptography==44.0.3
 + csvw==3.5.1
 + decorator==5.2.1
 + dlinfo==2.0.0
 + dnspython==2.7.0
 + 

## Echo Server from fastrtc.org [QuickStart](https://fastrtc.org/#__tabbed_1_1)

In [ ]:
%%writefile app.py
from fastrtc import Stream, ReplyOnPause
import numpy as np

def echo(audio: tuple[int, np.ndarray]):
    # The function will be passed the audio until the user pauses
    # Implement any iterator that yields audio
    # See "LLM Voice Chat" for a more complete example
    yield audio

stream = Stream(
    handler=ReplyOnPause(echo),
    modality="audio",
    mode="send-receive",
    # below rtc_configuration needed to work around potential firewall issues
    rtc_configuration={
        "iceServers": [{ "urls": ["stun:stun.l.google.com:19302"] }]
    }
)

Overwriting app.py


## Make it a Gradio App

In [ ]:
%%writefile -a app.py
stream.ui.launch()

Appending to app.py


## Running Gradio App

In [ ]:
!VIRTUAL_ENV=.venv GRADIO_SHARE="True" uv run app.py

/content/.venv/lib/python3.13/site-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/content/.venv/lib/python3.13/site-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/content/.venv/lib/python3.13/site-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/content/.venv/lib/python3.13/site-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):
silero_vad.onnx: 100% 1.81M/1.81M [00:00<00:00, 10.1MB/s]
INFO:	  Warming up VAD model.
INFO:	  VAD model warmed up.
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://048f4530b21e5e550a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from t

## FastRTC using FastAPI backend and HTML frontend

`nest-asyncio` and `pyngrok` are only needed for testing from within Google Colab.  
These can be excluded otherwise.

In [ ]:
%%bash
export VIRTUAL_ENV=.venv
uv add nest-asyncio pyngrok

Resolved 120 packages in 220ms
Prepared 2 packages in 66ms
Installed 2 packages in 4ms
 + nest-asyncio==1.6.0
 + pyngrok==7.2.8


In [ ]:
%%writefile app.py
from fastrtc import Stream, ReplyOnPause
import numpy as np
from fastapi import FastAPI
from fastapi.responses import HTMLResponse
import os

# Below imports are only needed to get this example to work within Google Colab
import nest_asyncio
from pyngrok import ngrok

# Below two lines are only needed to get this to work within Google Colab
NGROK_AUTH_TOKEN = os.getenv("NGROK_AUTH_TOKEN")
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

def echo(audio: tuple[int, np.ndarray]):
    # The function will be passed the audio until the user pauses
    # Implement any iterator that yields audio
    # See "LLM Voice Chat" for a more complete example
    yield audio

stream = Stream(
    handler=ReplyOnPause(echo),
    modality="audio",
    mode="send-receive",
    rtc_configuration={
        "iceServers": [{ "urls": ["stun:stun.l.google.com:19302"] }]
    }
)
app = FastAPI()
stream.mount(app)

# Optional: Add routes
@app.get("/")
async def _():
    return HTMLResponse(content=open("index.html").read())

# Below lines are only needed to get this example to work within Google Colab
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

Overwriting app.py


### Client Html Page

In [ ]:
%%writefile index.html
<html>

<head>
  <title>FastRTC Client Demo</title>
  <script src="https://cdn.jsdelivr.net/gh/lalanikarim/fastrtc-client@v0.1.2/fastrtc-client.js"></script>
</head>

<body>
  <h1>FastRTC Echo Server</h1>
  <button id="start">Connect</button>
  <button id="stop" style="display: none">Disconnect</button>
  <h3>Logs</h3>
  <pre class="logs"></pre>
  <audio></audio>
  <script defer>
    let logs = document.querySelector("pre")
    let startButton = document.querySelector("button#start")
    let stopButton = document.querySelector("button#stop")
    let client = FastRTCClient({
      additional_outputs_url: null,
      // below rtc_config is needed to work around potential firewall issues
      rtc_config: {
        iceServers: [
          {
            urls: ["stun:stun.l.google.com:19302"]
          }
        ]
      }
    })
    client.onConnecting(() => {
      logs.innerText += "Connecting to server.\n"
      startButton.style.display = "none"
      stopButton.style.display = "block"
    })
    client.onConnected(() => {
      logs.innerText += "Connected to server.\n"
    })
    client.onReadyToConnect(() => {
      logs.innerText += "Not connected to server.\n"
      startButton.style.display = "block"
      stopButton.style.display = "none"
    })
    client.onErrorReceived((error) => {
      logs.innerText += `serverError received: ${error}\n`
    })
    client.onPauseDetectedReceived(() => {
      logs.innerText += `pause detected event received. response will start now.\n`
    })
    client.onResponseStarting(() => {
      logs.innerText += `response starting event received. audio will start playing now.\n`
    })
    client.setShowErrorCallback((error) => {
      logs.innerText += `showError received: ${error}\n`
    })
    startButton.addEventListener("click", () => client.start())
    stopButton.addEventListener("click", () => client.stop())
  </script>
</body>

</html>

Writing index.html


### Testing webpage locally

In order to test the web client locally, you can run the below command.  
WebRTC will work locally over http over localhost.  
In order to use another domain, WebRTC will only work over https.

In [ ]:
from google.colab import userdata
ngrok_auth_token = userdata.get('NGROK_AUTH_TOKEN')

In [ ]:
!VIRTUAL_ENV=.venv NGROK_AUTH_TOKEN={ngrok_auth_token} uv run --with uvicorn uvicorn app:app

INFO:	  Warming up VAD model.
INFO:	  VAD model warmed up.
Public URL: https://c3af-34-106-222-65.ngrok-free.app
INFO:     Started server process [4516]
INFO:     Waiting for application startup.
INFO:	  Visit https://fastrtc.org/userguide/api/ for WebRTC or Websocket API docs.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     2600:1700:1f3:25f0:e0ce:4e3b:7d6f:68b4:0 - "GET / HTTP/1.1" 200 OK
INFO:     2600:1700:1f3:25f0:e0ce:4e3b:7d6f:68b4:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2600:1700:1f3:25f0:e0ce:4e3b:7d6f:68b4:0 - "POST /webrtc/offer HTTP/1.1" 200 OK
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [4516]
Task was destroyed but it is pending!
task: <Task pending name='Task-1348' coro=<WebRTCConnectionMixin.handle_offer.<locals>._() running at /content/.venv/lib/python3.13/site-packages/fastrtc/web

## Echo Server with STT and TTS

In [ ]:
%%writefile app.py
from fastrtc import Stream, ReplyOnPause, get_stt_model, get_tts_model
import numpy as np

stt_model = get_stt_model() # Moonshine
tts_model = get_tts_model() # Kokoro

def echo(audio: tuple[int, np.ndarray]):
    text = stt_model.stt(audio)
    for audio_chunk in tts_model.stream_tts_sync(text):
        yield audio_chunk

stream = Stream(
    handler=ReplyOnPause(echo),
    modality="audio",
    mode="send-receive",
    rtc_configuration={
        "iceServers": [{ "urls": ["stun:stun.l.google.com:19302"] }]
    }
)
stream.ui.launch()

Overwriting app.py


## Running Gradio App

In [ ]:
!VIRTUAL_ENV=.venv GRADIO_SHARE="True" uv run app.py

encoder_model.onnx: 100% 80.8M/80.8M [00:00<00:00, 170MB/s]
decoder_model_merged.onnx: 100% 166M/166M [00:00<00:00, 167MB/s]
INFO:	  Warming up STT model.
INFO:	  STT model warmed up.
kokoro-v1.0.onnx: 100% 326M/326M [00:02<00:00, 143MB/s]
voices-v1.0.bin: 100% 28.2M/28.2M [00:00<00:00, 50.7MB/s]
INFO:	  Warming up VAD model.
INFO:	  VAD model warmed up.
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://c2436baaab1e532aa9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c2436baaab1e532aa9.gradio.live


## LangChain App

In [ ]:
%%bash
export VIRTUAL_ENV=.venv
uv add langchain langchain-openai

Resolved 136 packages in 1ms
Audited 134 packages in 0.11ms


In [ ]:
%%writefile app.py
from fastrtc import Stream, ReplyOnPause, get_stt_model, get_tts_model
import numpy as np
from langchain.chat_models import init_chat_model

stt_model = get_stt_model() # Moonshine
tts_model = get_tts_model() # Kokoro

model = init_chat_model("openai:gpt-4.1-nano-2025-04-14")

def talk(audio: tuple[int, np.ndarray]):
    prompt = stt_model.stt(audio)
    response = model.invoke(prompt)
    for audio_chunk in tts_model.stream_tts_sync(response.content):
        yield audio_chunk

stream = Stream(
    handler=ReplyOnPause(talk),
    modality="audio",
    mode="send-receive",
    rtc_configuration={
        "iceServers": [{ "urls": ["stun:stun.l.google.com:19302"] }]
    }
)
stream.ui.launch()

Overwriting app.py


In [ ]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
!VIRTUAL_ENV=.venv OPENAI_API_KEY={api_key} GRADIO_SHARE="True" uv run app.py

INFO:	  Warming up STT model.
INFO:	  STT model warmed up.
INFO:	  Warming up VAD model.
INFO:	  VAD model warmed up.
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://6e81753dafb270a6d9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
/root/.local/share/uv/python/cpython-3.13.2-linux-x86_64-gnu/lib/python3.13/asyncio/base_events.py:750: RuntimeWarning: coroutine method 'aclose' of 'Kokoro.create_stream' was never awaited
  self._ready.clear()
Keyboard interruption in main thread... closing server.
Traceback (most recent call last):
  File "/content/.venv/lib/python3.13/site-packages/gradio/blocks.py", line 3019, in block_thread
    time.sleep(0.1)
    ~~~~~~~~~~^^^^^
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last)